In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.d_unit_conv
(
  material STRING,
  mat_unit STRING,
  denomintr FLOAT,
  eanupc STRING,
  ean_numtyp STRING,
  gross_wt FLOAT,
  height FLOAT,
  len FLOAT,
  numerator FLOAT,
  unit STRING,
  unit_dim STRING,
  unit_of_wt STRING,
  volume FLOAT,
  volumeunit STRING,
  width FLOAT
)
TBLPROPERTIES (
  'delta.autoOptimize.optimizeWrite' = 'true',
  'delta.autoOptimize.autoCompact' = 'true'
)
""")

In [0]:
spark.sql(f"""
CREATE OR REPLACE TEMP VIEW temp_d_unit_conv
AS
SELECT 
    material,
    mat_unit,
    denomintr,
    eanupc,
    ean_numtyp,
    gross_wt,
    height,
    len,
    numerator,
    unit,
    unit_dim,
    unit_of_wt,
    volume,
    volumeunit,
    width 
FROM (
    SELECT 
        *, ROW_NUMBER() OVER (PARTITION BY material, mat_unit ORDER BY proc_date DESC) rn 
    FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_0mat_unit
    WHERE end_date = '2400-01-01'
) a WHERE rn = 1
""")

In [0]:
# Load the temp table schema
df = spark.table(f"{catalog_name}.udp_wcm_gold_vcm_dwh.d_unit_conv")
columns = df.columns

# Create INSERT column list
insert_columns = ", ".join(columns)

# Create UPDATE SET clause
update_set_clause = ", ".join([f"main_tbl.{col} = temp_tbl.{col}" for col in columns])

# Construct the MERGE SQL
merge_sql = f"""
MERGE INTO {catalog_name}.udp_wcm_gold_vcm_dwh.d_unit_conv AS main_tbl
USING temp_d_unit_conv AS temp_tbl
ON main_tbl.material = temp_tbl.material AND main_tbl.mat_unit = temp_tbl.mat_unit
WHEN MATCHED THEN 
  UPDATE SET {update_set_clause}
WHEN NOT MATCHED THEN 
  INSERT ({insert_columns}) 
  VALUES ({insert_columns})
"""

# Run the SQL
spark.sql(merge_sql)